In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset

In [16]:
import pandas as pd
import numpy as np

def get_data(currency_lst,
             n_classes,
             frequency, 
             window_size,
             neutral_quantile = 0.25,
             beg_date = pd.Timestamp(2013,1,1),
             end_date = pd.Timestamp.now(),
             log_price = True,
             remove_trend = True,
             include_indicators = True,
             include_imfs = True,
             drop_open_high_low = True):
        
        X, y, dfs = {}, {}, {}     
        
        for cur in currency_lst:
            df = pd.read_csv(f"../data/0_raw/Binance/{str.lower(cur)}_usdt_1d.csv", index_col=0)
            df.index = pd.to_datetime(df.index, unit='s')
            df.sort_index(inplace=True)
            #df.index = df.Date.apply(pd.Timestamp)
            #df.sort_values("Date", inplace=True)
            #df.set_index("Date", inplace=True)
            df.drop(["Date"], axis=1, inplace=True)
            df.rename(str.lower, axis=1, inplace=True)
            
            if log_price:
                df[["close", "open", "high", "low"]] = df[["close", "open", "high", "low"]].apply(np.log, axis=1)
                   
            if n_classes == 3:
                df['pct_diff'] = df['close'].pct_change()
                neutral_quantiles = df['pct_diff'].abs().quantile(neutral_quantile)
                
                conditions = [(df['pct_diff'] < 0) & (df['pct_diff'].abs() > neutral_quantiles),
                              (df['pct_diff'] > 0) & (df['pct_diff'].abs() > neutral_quantiles)]

                classes = [0,1] # 2 is the default class if none of conditions is met, i.e. price change in the neutral range
            
                change_dir = np.select(conditions, classes, default=2)
            
            else:
                df['diff'] = df['close'].diff()
                change_dir = df['diff'].apply(lambda x: 0 if x <= 0 else 1)
            
            df.insert(loc=0, column="change_dir", value=change_dir)   
            df.dropna(inplace=True)       
            
            if include_indicators:
                from ta import add_all_ta_features
                indicators_df = add_all_ta_features(df, open="open", high="high", low="low", close="close", volume="volume", fillna=True)
                df[indicators_df.columns] = indicators_df
            else:
                df.drop(["volume", "open", "high", "low"], axis=1, inplace=True)
            
            if include_imfs:
                from PyEMD import EEMD
                eemd = EEMD()
                imfs = eemd(df["close"].values)
                imf_features = ["imf_"+str(i) for i in range(imfs.shape[0])]
                df = pd.concat((df, pd.DataFrame(imfs.T, columns=imf_features, index=df.index)), axis=1)
                
            if remove_trend:
                from statsmodels.tsa.seasonal import seasonal_decompose
                components = seasonal_decompose(df["close"], model="additive")
                df["close"] -= components.trend
                df.dropna(inplace=True)
                
            if drop_open_high_low:
                df.drop(["open", "high", "low"], axis=1, inplace=True)

            dfs[cur] = df
        
        min_dates = [df.index.min() for cur, df in dfs.items()]
        max_dates = [df.index.max() for cur, df in dfs.items()]
        beg_date = max([max(min_dates), beg_date])
        end_date = min([min(max_dates), end_date])
        common_range = pd.date_range(beg_date, end_date, freq=frequency)
        
        diff_col = 'pct_diff' if n_classes == 3 else 'diff'
        X = np.array([dfs[cur].loc[common_range].drop(["change_dir", diff_col], axis=1).values for cur in currency_lst])
        y = np.array([dfs[cur].loc[common_range, "change_dir"].values for cur in currency_lst])
        features = df.columns.tolist()
        
        return X, y, features, dfs

In [17]:
CURRENCY_LST = ['BTC', 'ETH', 'LTC']
N_CLASSES = 3
FREQUENCY = "D"
WINDOW_SIZE = 50
NEUTRAL_QUANTILE = 0.33

In [18]:
X, y, features, dfs = get_data(CURRENCY_LST,
                               N_CLASSES,
                               FREQUENCY, 
                               WINDOW_SIZE,
                               neutral_quantile = NEUTRAL_QUANTILE,
                               log_price=True,
                               remove_trend=True,
                               include_indicators = True,
                               include_imfs = False,
                               drop_open_high_low = True
                               )

C:\Users\Furkan\anaconda3\lib\site-packages\ta\trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
C:\Users\Furkan\anaconda3\lib\site-packages\ta\trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
C:\Users\Furkan\anaconda3\lib\site-packages\ta\trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
C:\Users\Furkan\anaconda3\lib\site-packages\ta\trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])
C:\Users\Furkan\anaconda3\lib\site-packages\ta\trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
C:\Users\Furkan\anaconda3\lib\site-packages\ta\trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._

In [19]:
(X.shape, y.shape)

((3, 1235, 85), (3, 1235))

In [20]:
dfs['BTC']

,change_dir,close,volume,pct_diff,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,...,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
Timestamp,,,,,,,,,,,,,,,,,,,,,
2017-08-21,0,-0.024659,691.743060,-0.002087,-1.172772e+02,4.223719e+02,-0.042802,-0.314014,30.377427,-195.987984,...,-66.771078,0.000000,8.310203,0.000000,-11.684041,-4.669418,-7.014624,-0.208662,-0.208880,-0.273291
2017-08-22,2,-0.024570,966.684858,0.000718,6.860528e+02,1.389057e+03,0.185084,0.553675,18.711425,-1401.361071,...,-31.378827,0.000000,8.307474,0.000000,-10.658271,-5.867188,-4.791083,0.071804,0.071778,-0.201684
2017-08-23,1,-0.013072,1001.136565,0.002186,4.953591e+02,2.390193e+03,0.105220,3.070887,41.990526,621.535735,...,-24.155995,-0.006036,8.313821,0.000000,-9.446604,-6.583071,-2.863532,0.218612,0.218373,0.016487
2017-08-24,1,0.027246,787.418753,0.005760,9.851933e+02,3.177612e+03,0.179280,8.024111,52.686021,181.229228,...,-5.098425,-0.002658,8.335488,0.000000,-10.171890,-7.300835,-2.871055,0.575970,0.574318,0.592553
2017-08-25,2,0.006414,573.612740,-0.000982,5.998016e+02,2.603999e+03,0.098832,6.204266,58.298126,238.421722,...,-14.692396,-0.000651,8.346212,0.000000,-12.552912,-8.351250,-4.201661,-0.098201,-0.098249,0.493770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-30,1,0.028502,68578.910045,0.006837,8.686301e+06,4.139648e+06,0.129211,598.624472,32.076290,4.026530,...,-27.949669,-0.050313,10.861746,0.248094,-0.817847,3.327408,-4.145255,0.683739,0.681412,31.753405
2021-05-01,2,0.035124,42600.351836,0.000168,8.691864e+06,4.182248e+06,0.131288,524.288158,36.660674,2.451554,...,-17.967363,-0.028570,10.866056,0.349755,-3.902179,1.881491,-5.783669,0.016759,0.016758,31.775486
2021-05-02,0,0.007162,36812.878863,-0.001949,8.676613e+06,4.145435e+06,0.140811,336.998825,40.019073,-1.149465,...,-14.880686,-0.019297,10.870946,0.024785,-7.180158,0.069161,-7.249319,-0.194909,-0.195099,31.518644


In [21]:
#the first WINDOW_SIZE values of change direction will be seen as NaN 

dfs['BTC']['change_dir'].value_counts() / len(dfs['BTC'])

1    0.367332
2    0.330377
0    0.302291
Name: change_dir, dtype: float64

In [8]:
N_CURRENCIES = 3
INPUT_FEATURE_SIZE = 1
WINDOW_SIZE = 50
TRAIN_PERCENTAGE, VAL_PERCENTAGE, TEST_PERCENTAGE = 0.70, 0.15, 0.15

In [9]:
class TimeSeriesDataset(Dataset):
    def __init__(self, 
                 x: np.ndarray,
                 y: np.ndarray,
                 data_use_type,
                 train_percentage = TRAIN_PERCENTAGE,
                 val_percentage = VAL_PERCENTAGE,
                 test_percentage = TEST_PERCENTAGE,
                 seq_len = WINDOW_SIZE, 
                 ):
        
        self.x = torch.tensor(x).float()
        self.y = torch.tensor(y).float()
        self.seq_len = seq_len
        
        self.data_use_type = data_use_type
        
        #self.train_size = int(len(self.x) * train_percentage)
        self.val_size = int(len(self.x) * val_percentage)
        self.test_size = int(len(self.x) * test_percentage)
        self.train_size = len(self.x) - self.val_size - self.test_size 
        
    def __len__(self):
        
        if self.data_use_type == "train":
            return self.train_size - self.seq_len
        
        elif self.data_use_type == "val":
            return self.val_size 
        
        else:
            return self.test_size
        
    def __getitem__(self, index):
        
        if self.data_use_type =="val":
            index = self.train_size + index - self.seq_len
            
        elif self.data_use_type =="test":
            index = self.train_size + self.val_size + index - self.seq_len
        
        window = self.x[index:index+self.seq_len]
        price_change = self.y[index+self.seq_len]
        
        return (window, price_change)

In [10]:
#datasets for the currency at 0 index of X
train, val, test = [TimeSeriesDataset(X[0], y[0], dtype) for dtype in ['train', 'val', 'test']]

In [11]:
len(train) + len(val) + len(test) + WINDOW_SIZE

1235

In [12]:
train[0]

(tensor([[-9.7841],
         [-9.7766],
         [-9.7373],
         [-9.6856],
         [-9.6378],
         [-9.5922],
         [-9.5783],
         [-9.5691],
         [-9.5564],
         [-9.5673],
         [-9.5577],
         [-9.5600],
         [-9.5566],
         [-9.5471],
         [-9.5420],
         [-9.5506],
         [-9.5742],
         [-9.6194],
         [-9.6427],
         [-9.6581],
         [-9.6554],
         [-9.6553],
         [-9.6369],
         [-9.6068],
         [-9.5806],
         [-9.5547],
         [-9.5410],
         [-9.5013],
         [-9.4577],
         [-9.4307],
         [-9.4050],
         [-9.3901],
         [-9.3678],
         [-9.3350],
         [-9.3336],
         [-9.3383],
         [-9.3410],
         [-9.3361],
         [-9.3207],
         [-9.3250],
         [-9.3314],
         [-9.3237],
         [-9.3097],
         [-9.2822],
         [-9.2504],
         [-9.2186],
         [-9.1654],
         [-9.0964],
         [-9.0548],
         [-9.0116]])

In [13]:
class MultiTimeSeriesDataset(Dataset):
    def __init__(self, 
                 n_currencies,
                 x: np.ndarray, 
                 y: np.ndarray,
                 data_use_type,
                 train_percentage = TRAIN_PERCENTAGE,
                 val_percentage = VAL_PERCENTAGE,
                 test_percentage = TEST_PERCENTAGE,
                 seq_len = WINDOW_SIZE, 
                 ):
        
        self.x = torch.tensor(x[:n_currencies]).float()
        self.y = torch.tensor(y[:n_currencies]).float()
        self.seq_len = seq_len
        self.data_use_type = data_use_type
        
        #self.train_size = int(len(self.x[0]) * train_percentage)
        self.val_size = int(len(self.x[0]) * val_percentage)
        self.test_size = int(len(self.x[0]) * test_percentage)
        self.train_size = len(self.x[0]) - self.val_size - self.test_size 
        
    def __len__(self):
        
        if self.data_use_type == "train":
            return self.train_size - ( self.seq_len)

        elif self.data_use_type == "val":
            return self.val_size
  
        else:
            return self.test_size
        
    
    def __getitem__(self, index):
        
        item = dict()
        
        if self.data_use_type =="val":
            index = self.train_size + index - self.seq_len
            
        elif self.data_use_type =="test":
            index = self.train_size + self.val_size + index - self.seq_len
        
        for i in range(N_CURRENCIES):
            item["currency_" + str(i) + "_window"] = self.x[i][index:index+self.seq_len]
            item["currency_" + str(i) + "_label"]  = self.y[i][index+self.seq_len]

        return item

In [14]:
train, val, test = [MultiTimeSeriesDataset(N_CURRENCIES, X, y, dtype) 
                    for dtype in ['train', 'val', 'test']]

In [15]:
#first day data for multi-task learning
train[0]

{'currency_0_window': tensor([[-9.7841],
         [-9.7766],
         [-9.7373],
         [-9.6856],
         [-9.6378],
         [-9.5922],
         [-9.5783],
         [-9.5691],
         [-9.5564],
         [-9.5673],
         [-9.5577],
         [-9.5600],
         [-9.5566],
         [-9.5471],
         [-9.5420],
         [-9.5506],
         [-9.5742],
         [-9.6194],
         [-9.6427],
         [-9.6581],
         [-9.6554],
         [-9.6553],
         [-9.6369],
         [-9.6068],
         [-9.5806],
         [-9.5547],
         [-9.5410],
         [-9.5013],
         [-9.4577],
         [-9.4307],
         [-9.4050],
         [-9.3901],
         [-9.3678],
         [-9.3350],
         [-9.3336],
         [-9.3383],
         [-9.3410],
         [-9.3361],
         [-9.3207],
         [-9.3250],
         [-9.3314],
         [-9.3237],
         [-9.3097],
         [-9.2822],
         [-9.2504],
         [-9.2186],
         [-9.1654],
         [-9.0964],
         [-9.0548],